### Process Data

This notebook process the input data.
It will be converted to python code to be shared with various models

In [71]:
# Pretty display for notebooks
%matplotlib inline

import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

# Import supplementary visualization code visuals.py
#import visuals as vs


#### Load the data from csv file
for now, you have to manualy unzip the file on your computer before running the following code

In [72]:

df_orig = pd.read_csv('./data/ny_hmda_2015.csv', low_memory=False, header=0, delimiter=",")

df = df_orig.copy()

#### Inspect the data

In [73]:
# group dataframe by unique combination of columns, and return a dataframe with count as the added last column
# df: the dataframe that contains all columns
# 
def get_count_of_unique_columns(df, list_of_columns):
    df_temp = df[list_of_columns]  #select the columns of list_of_columns
    df_count = df_temp.dropna().groupby(list_of_columns, as_index=False).size().reset_index().rename(columns={0:'count'})
    return df_count

In [74]:
# to inspect the dada
print ("dataframe head:")
#print(data.head())
display(df_orig.head(n=2))

num_rows = df_orig.shape[0]
num_col = df_orig.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))


df_action_count = get_count_of_unique_columns(df, ['action_taken','action_taken_name'])
print("df_action_count")
display(df_action_count)

df_ethnic_count = get_count_of_unique_columns(df, ['applicant_ethnicity', 'applicant_ethnicity_name'])
print("df_ethnic_count")
display(df_ethnic_count)

df_agency_count = get_count_of_unique_columns(df, ['agency_code', 'agency_abbr', 'agency_name'])
print("df_agency_count")
display(df_agency_count)


dataframe head:


,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,97.0,5,NaN,...,NY,New York,109000.0,187,363.0,1817.0,21.139999,5870.0,NaN,109.459999
1,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,200.0,5,NaN,...,NY,New York,71300.0,460,53.0,256.0,45.959999,3512.0,NaN,160.600006


Total number of records: 439654
Toatl numver of features: 78
df_action_count


,action_taken,action_taken_name,count
0,1,Loan originated,228054
1,2,Application approved but not accepted,14180
2,3,Application denied by financial institution,79697
3,4,Application withdrawn by applicant,39496
4,5,File closed for incompleteness,16733
5,6,Loan purchased by the institution,61490
6,7,Preapproval request denied by financial instit...,4


df_ethnic_count


,applicant_ethnicity,applicant_ethnicity_name,count
0,1,Hispanic or Latino,25073
1,2,Not Hispanic or Latino,320515
2,3,"Information not provided by applicant in mail,...",43358
3,4,Not applicable,50708


df_agency_count


,agency_code,agency_abbr,agency_name,count
0,1,OCC,Office of the Comptroller of the Currency,34741
1,2,FRS,Federal Reserve System,10211
2,3,FDIC,Federal Deposit Insurance Corporation,15555
3,5,NCUA,National Credit Union Administration,50944
4,7,HUD,Department of Housing and Urban Development,150441
5,9,CFPB,Consumer Financial Protection Bureau,177762


#### Some Helpful Commands 

In [75]:
# if read from zip file directly ...
#import zipfile
#with zipfile.ZipFile("./data/ny-home-mortgage.zip") as z:
#   with z.open("ny_hmda_2015.csv") as f:
#      train = pd.read_csv(f, low_memory=False, header=0, delimiter="\t")
#      print(train.head())    # print the first 5 rows


In [76]:
# to get all columns as a list
#list(data)

In [77]:
# To get portion of the dataframe:  data.loc[startrow:endrow,startcolumn:endcolumn]
# To slice the rows, and include all columns:  data.loc[0:5, :]
# To select all rows from a single column:    data.loc[: , "my_column_name"]

# first 3 rows, all columns
#data.loc[0:2, :]

In [78]:

drop_column = [
    #'action_taken',
 'action_taken_name',
 'agency_code',
 'agency_abbr',
 'agency_name',
 'applicant_ethnicity',
 'applicant_ethnicity_name',
 #'applicant_income_000s',
 'applicant_race_1',     #Need one-hot-encoding
 'applicant_race_2',     #Need one-hot-encoding
 'applicant_race_3',     #Need one-hot-encoding
 'applicant_race_4',     #Need one-hot-encoding
 'applicant_race_5',     #Need one-hot-encoding
 'applicant_race_name_1',
 'applicant_race_name_2',
 'applicant_race_name_3',
 'applicant_race_name_4',
 'applicant_race_name_5',
 'applicant_sex',          #Need one-hot-encoding
 'applicant_sex_name',
 'application_date_indicator',
 'as_of_year',
 'census_tract_number',
 'co_applicant_ethnicity', #Need one-hot-encoding
 'co_applicant_ethnicity_name',
 'co_applicant_race_1',     #Need one-hot-encoding
 'co_applicant_race_2',     #Need one-hot-encoding
 'co_applicant_race_3',     #Need one-hot-encoding
 'co_applicant_race_4',     #Need one-hot-encoding
 'co_applicant_race_5',     #Need one-hot-encoding
 'co_applicant_race_name_1',
 'co_applicant_race_name_2',
 'co_applicant_race_name_3',
 'co_applicant_race_name_4',
 'co_applicant_race_name_5',
 'co_applicant_sex',     #Need one-hot-encoding
 'co_applicant_sex_name',
 'county_code',         #Need one-hot-encoding
 'county_name',
 'denial_reason_1',   
 'denial_reason_2',
 'denial_reason_3',
 'denial_reason_name_1',
 'denial_reason_name_2',
 'denial_reason_name_3',
 'edit_status',         #?
 'edit_status_name',
 'hoepa_status',     #Need one-hot-encoding
 'hoepa_status_name',
 'lien_status',     #Need one-hot-encoding
 'lien_status_name',
 'loan_purpose',     #Need one-hot-encoding
 'loan_purpose_name',
 'loan_type',     #Need one-hot-encoding
 'loan_type_name',
 'msamd',     #Need one-hot-encoding
 'msamd_name',
 'owner_occupancy',      #Need one-hot-encoding
 'owner_occupancy_name',
 'preapproval',     #Need digitalize and one-hot-encoding
 'preapproval_name',
 #'property_type',       #Need one-hot-encoding
 'property_type_name',
 #'purchaser_type',       #Need one-hot-encoding
 'purchaser_type_name',
 'respondent_id',        #???
 'sequence_number',      #????
 'state_code',
 'state_abbr',
 'state_name',
 #'hud_median_family_income',
 #'loan_amount_000s',
 #'number_of_1_to_4_family_units',
 #'number_of_owner_occupied_units',
 #'minority_population',
 #'population',
 'rate_spread' ]  #???
 #'tract_to_msamd_income'  #???


# Make a copy of orignal data, so that you don't have to load the csv file again.
# However, this can take out memory, we may not need it later on.
# What we need is one-hot-encoding, and build a lookup dict for the columns that is been one-hot-encoded


# Drop the columns in the drop_column list
df.drop(drop_column, axis=1, inplace=True)

# df is altered after dropping the column. Inspect the data again.
df.loc[3000:3002, :]



,action_taken,applicant_income_000s,property_type,purchaser_type,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,tract_to_msamd_income
3000,1,195.0,1,1,71300.0,179,224.0,672.0,29.52,8651.0,158.070007
3001,1,62.0,1,9,68500.0,248,1187.0,813.0,4.57,2541.0,104.769997
3002,1,18.0,1,0,66400.0,10,1588.0,1006.0,11.45,3475.0,88.849998


Other commands that may be helpful when pre-procses the dataframe

```python

df.replace('n/a', np.nan,inplace=True)
df.emp_length.fillna(value=0,inplace=True)

df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'] = df['emp_length'].astype(int)

df['term'] = df['term'].apply(lambda x: x.lstrip())
'''
